In [19]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

In [20]:
data = pd.read_csv('20231031.csv',encoding='utf-8')
data = data[~data['仓库'].isin(['8013','8021','8025','8031'])]
# data =data[data['零件号']!='6RD959801E']
data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
new_df = data.groupby(['零件号','日期','仓库'])['需求数量'].sum().reset_index()
new_df['日期'] = pd.to_datetime(new_df['日期'])

In [21]:
new_df.loc[(new_df['零件号'] == '6RD959801E') & (new_df['日期'] == '2023/3/27')& 
           (new_df['仓库'] == '1000-1'), '需求数量'] = 6

In [22]:
data_des = new_df.copy()
data_des["year"] = pd.to_datetime(data_des['日期']).dt.year.astype(int)
data_des["month"] = pd.to_datetime(data_des['日期']).dt.month.astype(int)
real_sum_counts = data_des.groupby(['零件号','仓库','year','month']).sum().reset_index()
real_sum_counts.sort_values(by=['零件号','仓库','year','month']).head(2)

,零件号,仓库,year,month,需求数量
0,11D941078C,1000-2,2023,1,1
1,11D941078C,1000-2,2023,2,3


In [23]:
# real_sum_counts[(real_sum_counts['零件号']=='3CC945208A')&(real_sum_counts['仓库']=='1000-1')].tail(10)

In [24]:
# 生成日期范围
date_range = pd.date_range(start='2022-12-01', end='2023-10-31', freq='D')
# 创建空的DataFrame，准备存储填充后的结果
filled_df = pd.DataFrame()

# 针对每个 SKU 进行填充操作
for sku, group in new_df.groupby(['零件号','仓库']):
    sku_group = group.set_index('日期').reindex(date_range, fill_value=0).reset_index()
    sku_group['零件号'] = sku[0]
    sku_group['仓库'] = sku[1]
    filled_df = filled_df.append(sku_group, ignore_index=True)

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/2866502645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/2866502645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/2866502645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filled_df = filled_df.append(sku_group, ignore_index=True)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/2866502645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [25]:
import calendar  
import datetime  
  

    
def feature_processing(data,end_time):
    data['合并列'] = data['零件号'] + '_' + data['仓库']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index <= end_time].copy()
    return data_train


def demods_groby_month(data):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns =['year','month','零件号','pred_values','仓库']
    return data_prs_info


def find_outliers_3sigma(data):  
    
    mean = sum(data) / len(data)  
    std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
    if mean<=150:
        thr_sig_num_high  = mean + 3 * std_dev
    else:
        thr_sig_num_high  = mean
        
    thr_sig_num_low  = mean - 1 * std_dev
#     q3 = lambda x: x.quantile(0.75)  
#     q1 = lambda x: x.quantile(0.25)  
#     thr_sig_num_high = q3(x) + 1.5 * (q3(x) - q1(x)) 
#     thr_sig_num_low = q1(x) - 1.5 * (q3(x) - q1(x)) 

    return thr_sig_num_high,thr_sig_num_low



# 生成一个日期对象，表示2023年1月1日  
start_date = datetime.datetime(2023, 1, 1)  
  
# 初始化一个空列表来保存每个月的最后一天  
month_end_dates = []  
  
# 使用 calendar.monthrange() 函数获取每个月的天数  
# 然后加1天，因为我们想要的是当月的最后一天  
for month in range(2, 8):  # 这里 13 是因为一年只有12个月  
    _, num_days = calendar.monthrange(2023, month)  # 获取2023年的日历  
    end_date = start_date.replace(month=month, day=num_days)  # 创建日期对象  
    month_end_dates.append(end_date)  # 添加到列表中  
pred_full_pp = pd.DataFrame()

for date in month_end_dates:  
    end_train = date.strftime("%Y-%m-%d")
    data_train = feature_processing(filled_df,date)
    sku_list = data_train.columns
    # 创建空的字典，用于存储每个时间序列的预测模型
    forecasters = {}
    # 循环遍历每批次SKU，分别训练预测模型
    for i in range(0, len(sku_list), 200):
        batch_skus = sku_list[i:i + 200]
        for sku in batch_skus:
            forecaster = ForecasterAutoreg(  
            regressor=Ridge(random_state=123),  
            lags=30,  
                    )
            # 拟合模型
            forecaster.fit(y=data_train[sku])
            forecasters[sku] = forecaster
        # print(f"Finished training batch {i // self.batch_size + 1}/{len(sku_list) // self.batch_size + 1}")
    # 进行未来预测
    predictions = pd.DataFrame()
    for sku, forecaster in forecasters.items():
        forecast = forecaster.predict(steps=30)
        predictions[sku] = forecast
    predictions[predictions < 0.1] = 0


    his_info = demods_groby_month(data_train)
    thr_std = his_info.groupby(['零件号','仓库'])['pred_values'].apply(find_outliers_3sigma).reset_index()
    thr_std['pred_values_high']= [x[0] for x in thr_std['pred_values']]
    thr_std['pred_values_lower']= [x[1] for x in thr_std['pred_values']]
    for i in ['pred_values_high','pred_values_lower']:
        thr_std[i] =[0 if x<0 else x for x in thr_std[i]]
    thr_std = thr_std.drop(['pred_values'],axis =1 )
    
    

    pred_info = demods_groby_month(predictions)
    pred_info['pred_values'] = pred_info['pred_values'].round(2)
    
    
    full_pred_info = pd.merge(pred_info,thr_std,on =['零件号','仓库'],how ='left')
    full_pred_info['pred_values'] = full_pred_info.apply(lambda row: row['pred_values_high'] 
                                if row['pred_values'] > row['pred_values_high'] 
                                 else (row['pred_values_lower'] if row['pred_values'] < row['pred_values_lower'] 
                                       else row['pred_values']), axis=1)  
#     full_pred_info = full_pred_info.drop(['pred_values_lower','pred_values_high'],axis =1 )
    

    pred_full_pp = pred_full_pp.append(full_pred_info)


pred_full_pp.head()



/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_full_pp = pred_full_pp.append(full_pred_info)
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/3861897121.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower
0,2023,3,11D941078C,5.074991,1000-2,5.074991,0.086114
1,2023,3,3CC945208A,2.000000,1000-1,5.408324,0.419448
2,2023,3,3CC945208A,0.000000,3001,1.747547,0.000000
3,2023,3,3CC945208A,0.000000,5001,1.747547,0.000000
4,2023,3,3CC945208A,0.000000,5002,0.000000,0.000000


In [26]:
compar_pred_real_info = pd.merge(pred_full_pp,real_sum_counts,on =['零件号','仓库','year','month'],how ='left')
compar_pred_real_info.fillna(0,inplace=True)

In [27]:
svg_pred =pd.read_csv('svg预测结果.csv',encoding='gb18030', header=1)
svg_pred['日期'] = pd.to_datetime(svg_pred['日期'], format='%Y%m').dt.strftime('%Y-%m')
svg_pred["year"] = pd.to_datetime(svg_pred['日期']).dt.year.astype(int)
svg_pred["month"] = pd.to_datetime(svg_pred['日期']).dt.month.astype(int)
svg_pred = svg_pred.groupby(['零件代码','仓库','year','month'])['预测值'].sum().reset_index()
svg_pred = svg_pred.rename(columns ={"零件代码":"零件号"})

In [28]:
full_compart_info  = pd.merge(compar_pred_real_info,svg_pred,on = ['零件号','仓库','year','month'],how ='left')

In [29]:
full_compart_info.fillna(0,inplace =True)

full_compart_info['需求数量']= [1.01 if x ==0 else x for x in full_compart_info['需求数量']]

full_compart_info['pred_values']= [1 if x ==0 else x for x in full_compart_info['pred_values']]

full_compart_info['chumi_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['pred_values']) 
                                 / full_compart_info['需求数量']) 
full_compart_info['svg_mape'] = (abs(full_compart_info['需求数量'] - full_compart_info['预测值']) 
                                 / full_compart_info['需求数量']) 
full_compart_info.sort_values(['零件号']).tail(50)

,year,month,零件号,pred_values,仓库,pred_values_high,pred_values_lower,需求数量,预测值,chumi_mape,svg_mape
54,2023,7,3CC945208A,1.000000,8011,2.613866,0.000000,2.00,0.833333,0.500000,0.583333
55,2023,7,3CC945208A,1.000000,8012,2.385277,0.000000,1.00,2.000000,0.000000,1.000000
56,2023,7,3CC945208A,0.150000,8015,2.813848,0.014432,4.00,1.066667,0.962500,0.733333
57,2023,7,3CC945208A,1.000000,8022,1.913186,0.000000,4.00,0.428571,0.750000,0.892857
58,2023,7,3CC945208A,2.450000,8023,9.642465,0.976321,13.00,3.464286,0.811538,0.733516
66,2023,8,3CC945208A,0.530000,8011,3.195870,0.000000,4.00,1.300000,0.867500,0.675000
61,2023,8,3CC945208A,6.800000,1000-1,18.066575,0.811142,8.00,7.000000,0.150000,0.125000
62,2023,8,3CC945208A,9.800000,3001,9.982123,0.000000,5.00,3.904762,0.960000,0.219048
63,2023,8,3CC945208A,2.780000,5001,8.388735,0.037088,9.00,3.142857,0.691111,0.650794
70,2023,8,3CC945208A,10.600000,8023,15.890614,0.536462,14.00,5.583333,0.242857,0.601191


In [30]:
full_compart_info.sort_values(['零件号','month']).to_excel('预测效果_20231101.xlsx')

In [31]:
full_compart_info.chumi_mape.mean()

1.1747359173145018

In [32]:
full_compart_info.svg_mape.mean()


# 1.提高模型响应速度 最近月份的比重
# 2.对话机器人的流程 chatgpt 

1.5952113465501807

In [33]:
full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_4755/433864545.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_compart_info.groupby(['month'])['chumi_mape','svg_mape'].mean().reset_index()


,month,chumi_mape,svg_mape
0,3,0.281847,0.798175
1,4,1.102471,1.800608
2,5,0.523727,1.039344
3,6,0.470545,0.952562
4,7,3.275545,3.789389
5,8,1.394281,1.191190


In [34]:

def feature_processing(data,end_time):
    data['合并列'] = data['零件号'] + '_' + data['仓库']
    # 去除多余的列名
    data = data.drop(columns=['零件号', '仓库'])
    data = data.set_index(['index', '合并列'])['需求数量'].unstack()
    data.columns.name = None
    data = data.reset_index()
    new_df = data.copy().rename(columns={'index': 'date'})
    new_df['date'] = pd.to_datetime(new_df['date'], format='%Y-%m-%d')
    new_df = new_df.set_index('date')
    new_df = new_df.asfreq('D')
    new_df = new_df.sort_index()
    data_train = new_df[new_df.index <= end_time].copy()
    return data_train


def demods_groby_month(data):
    """天级别聚合month"""
    data_prs = data.reset_index()
    data_prs = data_prs.rename(columns={'index': 'date'})
    data_prs["year"] = pd.to_datetime(data_prs['date']).dt.year.astype(int)
    data_prs["month"] = pd.to_datetime(data_prs['date']).dt.month.astype(int)
    data_prs_info = data_prs.groupby(['year', 'month']).sum().reset_index()
    data_prs_info = data_prs_info.set_index(['year', 'month']).stack()
    data_prs_info = data_prs_info.rename_axis(index=['year', 'month', '零件号'])
    data_prs_info = data_prs_info.reset_index()
    data_prs_info[['零件号', '仓库代码']] = data_prs_info['零件号'].str.split('_', expand=True).reset_index(drop=True)
    data_prs_info.columns =['year','month','零件号','pred_values','仓库']
    return data_prs_info


def find_outliers_3sigma(data):  

    mean = sum(data) / len(data)  
    std_dev = (sum((x - mean) ** 2 for x in data) / len(data)) ** 0.5  
    thr_sig_num_high  = mean + 3 * std_dev
    thr_sig_num_low  = mean - 1 * std_dev

#     q3 = lambda x: x.quantile(0.75)  
#     q1 = lambda x: x.quantile(0.25)  
#     thr_sig_num_high = q3(x) + 1.5 * (q3(x) - q1(x)) 
#     thr_sig_num_low = q1(x) - 1.5 * (q3(x) - q1(x)) 

    return thr_sig_num_high,thr_sig_num_low

test  = feature_processing(filled_df,'2023-03-31')

test = demods_groby_month(test)


output = his_info.groupby(['零件号','仓库'])['pred_values'].apply(find_outliers_3sigma).reset_index()
output['pred_values_high']= [x[0] for x in output['pred_values']]
output['pred_values_lower']= [x[1] for x in output['pred_values']]
output


,零件号,仓库,pred_values,pred_values_high,pred_values_lower
0,11D941078C,1000-2,"(7.197905971925756, 0.43403134269141463)",7.197906,0.434031
1,3CC945208A,1000-1,"(18.066575445053047, 0.811141518315651)",18.066575,0.811142
2,3CC945208A,3001,"(9.982122564908861, -0.49404085496962047)",9.982123,-0.494041
3,3CC945208A,5001,"(8.388734908183775, 0.0370883639387416)",8.388735,0.037088
4,3CC945208A,5002,"(1.1171567416492216, -0.20571891388307384)",1.117157,-0.205719
5,3CC945208A,6000,"(6.103570079844435, -0.20119002661481145)",6.103570,-0.201190
6,3CC945208A,8011,"(3.1958704751503912, -0.2319568250501305)",3.195870,-0.231957
7,3CC945208A,8012,"(2.462911636061258, -0.3209705453537527)",2.462912,-0.320971
8,3CC945208A,8015,"(4.930834336909582, -0.1436114456365274)",4.930834,-0.143611
9,3CC945208A,8022,"(4.680834336909582, -0.3936114456365274)",4.680834,-0.393611


In [35]:
test[(test['零件号']=='3CC945208A')&(test['仓库']=='5001')]

,year,month,零件号,pred_values,仓库
3,2022,12,3CC945208A,0,5001
15,2023,1,3CC945208A,0,5001
27,2023,2,3CC945208A,1,5001
39,2023,3,3CC945208A,1,5001


In [36]:
real_sum_counts[(real_sum_counts['零件号']=='3CC945208A')&(real_sum_counts['仓库']=='5001')]

,零件号,仓库,year,month,需求数量
28,3CC945208A,5001,2023,2,1
29,3CC945208A,5001,2023,3,1
30,3CC945208A,5001,2023,4,6
31,3CC945208A,5001,2023,5,4
32,3CC945208A,5001,2023,6,1
33,3CC945208A,5001,2023,7,4
34,3CC945208A,5001,2023,8,9
35,3CC945208A,5001,2023,9,18
36,3CC945208A,5001,2023,10,13
